In [ ]:
# This scraper will read the lab_outcomes.csv file and loop through each unique slug and append that slug to the base url on the outreachdashboard. 
# This will download the overview spreadsheet of each campaign/event. The directory containing lab_outcomes sheet should be specified and an output folder should also be specified.
# An origincal copy wil be stored as .csv in the output directory and another copy with some additional columns added for processing purposes will be saved as .xlsx file.  The files are named after their respective slug.
# A list of failed retrievals is also generated of a spreadsheet with a matching slug title was not found on the outreach dashboard. Generally this is due to a mismatch with the name of the event/campaign in the pulled data vs. the name on the outreach dashboard.
# The files are then uploaded to the google drive and the shareable link to the file is writen into the spreadsheet.

In [ ]:
import pandas as pd
import requests
import os
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.http import MediaFileUpload
from google.oauth2.credentials import Credentials
from io import StringIO


In [ ]:
CLIENT_SECRET_FILE = 'client_secrets.json'


def get_shared_link(file_id):
    return f"https://drive.google.com/file/d/{file_id}/view"


def upload_to_drive(filename, folder_id, drive_id='0AEEIqqbs6jViUk9PVA'):
    # Load credentials or prompt user to login if not available
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json')
    else:
        flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, ['https://www.googleapis.com/auth/drive.file'])
        creds = flow.run_local_server(port=8080)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('drive', 'v3', credentials=creds)
    
    file_metadata = {'name': os.path.basename(filename), 'parents': [folder_id]}
    media = MediaFileUpload(filename, resumable=True)
    
    try:
        file = service.files().create(
            body=file_metadata,
            media_body=media,
            fields='id',
            supportsAllDrives=True, 
        ).execute()
       

        # Get shared link
        shared_link = get_shared_link(file.get('id'))
        return shared_link
    except Exception as error:
        print(f"An error occurred: {error}")
        return None




In [ ]:
# Load spreadsheet.
xl = pd.ExcelFile('lab_outcomes.xlsx')

# Load desired xl sheet into a dataframe by its name.
df = xl.parse('courses_filtered.tsv')

In [ ]:
# Modify slug text string so that it can be used for file name.
# Create base url.
# Setup download directory.
# Create iteratble list of slugs that will be appended to base url inside subsequent loop.
df['slug2'] = df['slug'].replace('/', '_', regex=True)
mydir = '/output'
base_url = 'https://outreachdashboard.wmflabs.org/course_csv?course='
slugs = df['slug'][0:]
df['Notes/file location'] = df['Notes/file location'].astype('object')


In [ ]:
# Initial conditions
current_slug = ''
SHARED_DRIVE_ID = '1CYd5RhWcKr8IBJ5w68EcXvr6hcNdYgJP'


# Loop through xl sheet.
for index, row in df.iterrows():
    
    new_slug = row['slug']

    # Avoid duplicates.
    if current_slug == new_slug:
        continue

    url = base_url + row['slug']
    response = requests.get(url)

    # Successful request.
    if response.status_code == 200:
        current_slug = row['slug']

        # Write to a file.
        content = response.content.decode('utf-8')
        df_temp = pd.read_csv(StringIO(content))

       # Modify DataFrame.
        if row[0] == 'Event':
            df_temp.insert(1, 'campaign title', 'NA')
        else:
            df_temp.insert(1, 'campaign title', row['title'])
        
        df_temp.insert(0, 'organizers', row['facilitators'])

        # Save as Excel.
        filename = f"{row['slug2']}.xlsx"
        df_temp.to_excel(filename, index=False)

        
        # Upload to Google Drive.
        file_url = upload_to_drive(filename, SHARED_DRIVE_ID)

        # Save the URL.
        df.at[index, 'Notes/file location'] = file_url

        print(df['Notes/file location'])
    
    else:
        print(f'Failed to download file from {url}')